In [7]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library
import lxml

In [4]:
!pip install lxml

     |████████████████████████████████| 4.5 MB 1.8 MB/s            


In [2]:
import sys
!{sys.executable} -m pip install wikipedia

  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
    Running setup.py install for wikipedia ... done


In [13]:
! pip install lxml

In [ ]:
# transform the data into a pandas dataframe

df = pd.read_csv('lisboa_data.csv')
df.head(3)

In [17]:
url = 'https://raw.githubusercontent.com/mdsl22/Coursera_Capstone/master/Geo_Lisbon.csv'


In [19]:
df_geo = pd.read_csv(url)
df_geo.head(3)

# Create Latitude and Longitude columns in the dataframe lisbon
lisbon_df['Latitude'] = np.nan
lisbon_df['Longitude'] = np.nan

NameError: name 'lisbon_df' is not defined